In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('train_comment_small_100.csv')

In [5]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [9]:
dataset

,comment_text,toxic
0,"""\n \n I'm sure if you worked on articles othe...",0.0
1,D'aww! He matches this background colour I'm s...,0.0
2,"Hey man, I'm really not trying to edit war. It...",0.0
3,"""\n More\n I can't make any real suggestions o...",0.0
4,REDIRECT Talk:Voydan Pop Georgiev- Chernodrinski,0.0
...,...,...
95,Because I find it funny - I was blocked five m...,0.0
96,"""\n \n Hi \n \n He has made veiled threats ag...",0.0
97,"(To ), Actually, they did noy mess anything up...",0.0
98,"""Citation 2 has reported false information on ...",0.0


In [11]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []

# THERE IS AN ERROR IN THE LINE BELOW, PLEASE UPDATE THE UPPER LIMIT FOR THE 
# RANGE.
for i in range(0, dataset.shape[0]-1):
    review = re.sub('[^a-zA-Z]', ' ', dataset['comment_text'][i])
    review = review.lower()
    review = review.split()
    
# THERE IS AN ERROR IN THE LINE BELOW, THE LINE BELOW THAT I HAVE COMMENTED 
# NEEDS TO BE COMMENTED IN THE ORIGINAL NOTEBOOK.  
#     review.remove('united')
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wujiangu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:,0].values
y1 = y[:99]
y1

array(['"\n \n I\'m sure if you worked on articles other than Arcayne you\'d do fine. You two seem to have a problem with each other, so why do you seek him out and edit where he does? â€“xenotalk "',
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       '"\n More\n I can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents"" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me k

In [24]:
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y1)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [26]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [30]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()

classifier.add(Dense(output_dim = 20, units = 'uniform', activation = 'relu', input_dim = 20))

classifier.add(Dense(output_dim = 20, init = 'uniform', activation = 'relu'))

classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'softmax'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 3, nb_epoch = 5)

TypeError: ('Keyword argument not understood:', 'output_dim')

In [ ]:
# THERE WAS AN ERROR IN THE FOLLOWING CODE THAT I HAVE COMMENTED AND THE CORRECT
# CODE IS BELOW IT. PLEASE FIX IT IN THE ORIGINAL NOTEBOOK

# scores = classifier.evaluate(vectorizer.transform(X_test), y_test, verbose=1)
scores = classifier.evaluate(X_test, y_test, verbose=1)
print("Accuracy:", scores[1])

# THE FOLLOWING CODE WAS MISSING, PLEASE ADD IT IN THE ORIGINAL NOTEBOOK
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)